In [8]:
import os as os
from os import *
import random
import shutil

In [3]:
PATH = 'data'
train_root = f'{PATH}/train'
valid_root = f'{PATH}/valid'
classes = os.listdir(train_root)
print(classes)

['wrapsnslings', 'womensleevelesstop', 'womencasualshoes', 'BabyBibs', 'womanshirtsleeve', 'womenplussizedtop', 'womenstripedtop', 'BabyHat', 'womenlazyshoes', 'womenknittedtop', 'womenpointedflatshoes', 'BabyShirt', 'womenlongsleevetop', 'womendollshoes', 'BabyPants', 'womenchiffontop', 'womenpeashoes', 'PackageFart']


In [32]:
def mv_to_valid(chosen_files, classname):
    for i in chosen_files:
        shutil.move(f'{train_root}/{classname}/{i}', f'{valid_root}/{classname}/{i}')

In [37]:
def train_val_split(val_percentage=0.2):
    random.seed(1)
    for classname in classes:
        os.makedirs(f'{valid_root}/{classname}', exist_ok=True)
        list_of_imgs = !ls "{train_root}/{classname}"
        random.shuffle(list_of_imgs)
        n_files_moved=int(len(list_of_imgs)*val_percentage)
        selected_files = [list_of_imgs[m] for m in range(n_files_moved)]
        mv_to_valid(selected_files, classname)

In [38]:
def mv_to_train(chosen_files, classname):
    for i in chosen_files:
        src = f'{valid_root}/{classname}/{i}'
        dst = f'{train_root}/{classname}/{i}'
        shutil.move(src, dst)
    
def put_back_to_train():
    for classname in classes:
        list_of_file_dirs = !ls "{PATH}/valid/{classname}"
        mv_to_train(list_of_file_dirs, classname)
    

In [26]:
put_back_to_train()

In [39]:
train_val_split()

In [11]:
sample_train = f'{PATH}/sample/train'
sample_valid = f'{PATH}/sample/valid'
os.makedirs(sample_train, exist_ok=True)
os.makedirs(sample_valid, exist_ok=True)
random.seed(1)

def cp_to_dir(filenames, src_root, dest_root):
    for fn in filenames:
        src = f'{src_root}/{fn}'
        dst = f'{dest_root}/{fn}'
        shutil.copy2(src, dst)

def build_sample_train(perc=0.1):
    classes = os.listdir(train_root)
    for classname in classes:
        dest_root = f'{sample_train}/{classname}'
        src_root = f'{train_root}/{classname}'
        os.makedirs(dest_root, exist_ok=True)
        imgs = os.listdir(src_root)
        random.shuffle(imgs)
        n_to_copy = int(len(imgs)*perc)
        selected = [imgs[i] for i in range(n_to_copy)]
        cp_to_dir(selected, src_root, dest_root)

In [13]:
def build_sample_valid(perc=0.1):
    classes = os.listdir(valid_root)
    for classname in classes:
        dest_root = f'{sample_valid}/{classname}'
        src_root = f'{valid_root}/{classname}'
        os.makedirs(dest_root, exist_ok=True)
        imgs = os.listdir(src_root)
        random.shuffle(imgs)
        n_to_copy = int(len(imgs)*perc)
        selected = [imgs[i] for i in range(n_to_copy)]
        cp_to_dir(selected, src_root, dest_root)

In [12]:
build_sample_train()

In [14]:
build_sample_valid()